In [2]:
pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.3 MB/s eta 0:00:00


marital status of us people. table 1. 0.51 married, 0.34 single, 0.15 divorced
https://www.census.gov/data/tables/time-series/demo/families/marital.html

High sch and lower 28.9 + 3.3 + 6.1 +16.5(0.377)= 44.4%

Bachelor/associate degree 22.2 + 9.9 + 16.5(0.623)= 42.4%

Masters's 9.9%, PhD 3.3%
https://educationdata.org/education-attainment-statistics

only 62.3% of people who started college graduated within 6 years
https://finance.yahoo.com/news/college-graduation-statistics-035013402.html

for banks < 1b in assets, median marketing cost is 440,000 and mean marketing cost is 446,672
https://bankingjournal.aba.com/2025/01/bank-marketing-budget-and-staffing-considerations-for-2025/


46 monthly payments on average
https://www.frbservices.org/news/fed360/issues/090324/cash-consumer-diary-reveals-increased-payments


US consumers made an average transaction amount of approximately $140.
https://www.frbsf.org/research-and-insights/publications/fed-notes/2021/05/2021-findings-from-the-diary-of-consumer-payment-choice/


cost per reach (CPR) for bank marketing for the following mediums

sms: $0.01–0.05 https://www.webfx.com/blog/digital-marketing/sms-marketing-pricing

email: 0.01 - 0.1

direct mail: 0.8 - 1.6 https://www.emarketer.com/content/banking-customers-enjoy-direct-mail-marketing

social media: 0.01 to 0.05

"In 2018, banks generated, on average, $18.34 for every dollar spent on marketing". this is only for the 210 banks surveyed
https://thefinancialbrand.com/news/bank-marketing/bank-marketing-budgets-advertising-roi-strategy-88835

In [4]:
import pandas as pd
import numpy as np
from faker import Faker  # Import the Faker library

# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
np.random.seed(42)
Faker.seed(42)  # Seed for Faker to ensure reproducibility

#number of customers and campaigns
num_customers = 10000
num_campaigns = 5

#generating customer id
customer_ids = set()  #set to ensure uniqueness

while len(customer_ids) < num_customers:
    customer_ids.add(fake.uuid4())  #only unique values

customer_ids = list(customer_ids)

#generating age
ages = np.random.randint(18, 70, size=num_customers)

#generate gender (male, female)
genders = np.random.choice(['male', 'female'], size=num_customers, p=[0.5, 0.5])  #50% male, 50% female

#generate marital status (single, married, divorced)
marital_statuses = np.random.choice(['single', 'married', 'divorced'], size=num_customers, p=[0.34, 0.51, 0.15])

#education level
education = np.random.choice(['high school', 'bachelor', 'master', 'phd'], size=num_customers, p=[0.444, 0.424, 0.099, 0.033])

#generate monthly income based on age with variability and outliers
base_income = 2000  #base monthly income for the youngest age (18)
min_age = 18
age_factor = 100  #let income increases by $100 for each year of age
random_noise = np.random.normal(0, 500, size=num_customers)  #random noise for variability

#calculate monthly income
monthly_incomes = base_income + (ages - min_age) * age_factor + random_noise
monthly_incomes = np.round(monthly_incomes, 2)  #2dp

#introduce outliers (1% chance of earning up to $30,000)
outlier_probability = 0.01  #1% chance of being an outlier
outlier_mask = np.random.rand(num_customers) < outlier_probability  #identify the outliers
monthly_incomes[outlier_mask] = np.random.uniform(20000, 30000, size=np.sum(outlier_mask))  #outlier incomes

#ensure no negative incomes
monthly_incomes = np.clip(monthly_incomes, 0, None) #no minimum income

#generate bank balance (stronger trend with income and age)
#bank balance = (income * income_balance_factor) + (age * age_balance_factor) + random noise

income_balance_factor = 12  #bank balance is roughly 12 times the monthly income (simulating savings)
age_balance_factor = 500  #bank balance increases by $500 for each year of age
random_balance_noise = np.random.normal(0, 5000, size=num_customers)  #random noise for variability

#calculate bank balance
bank_balances = (monthly_incomes * income_balance_factor) + (ages * age_balance_factor) + random_balance_noise
bank_balances = np.round(bank_balances, 2)  # Round to 2 decimal places

#no negative bank balances
bank_balances = np.clip(bank_balances, 0, None)

#generate transaction history
#avg_transaction_value: normal distribution with mean = 140, std = 30
avg_transaction_values = np.random.normal(140, 30, size=num_customers)
avg_transaction_values = np.round(avg_transaction_values, 2)  #2dp
avg_transaction_values = np.clip(avg_transaction_values, 0, 500) #possible to be 0 espeically if u dun use that bank anymore

#transaction_frequency: Normal distribution with mean = 46, std = 10
transaction_frequencies = np.random.normal(46, 10, size=num_customers)
transaction_frequencies = np.round(transaction_frequencies, 0).astype(int)  #round to nearest integer
transaction_frequencies = np.clip(transaction_frequencies, 0, 100)  #again, possible to be 0 espeically if u dun use that bank anymore

#generate campaign data
campaign_ids = np.arange(1, num_campaigns + 1) #list of possible ids
mediums = np.random.choice(['email', 'SMS', 'direct mail', 'social media'], size=num_customers)  #the medium of campaign for each customer
personalization_levels = np.random.choice(['low', 'medium', 'high'], size=num_customers)

#generate cost per communication for each medium
cost_per_communication = {
    'email': np.random.uniform(0.01, 0.10, size=num_customers),  #email cost: $0.01–$0.10
    'SMS': np.random.uniform(0.01, 0.05, size=num_customers),  #sms cost: $0.01–$0.05
    'direct mail': np.random.uniform(0.80, 1.60, size=num_customers),  #direct mail cost: $0.80–$1.60
    'social media': np.random.uniform(0.01, 0.05, size=num_customers)  #social media cost: $0.01–$0.05
}

#calculate total campaign cost for each campaign
campaign_costs = {campaign_id: 0 for campaign_id in campaign_ids}  #initialization
for i in range(num_customers):
    medium = mediums[i]
    campaign_id = campaign_ids[i % num_campaigns]  #assign customers to campaigns in a round-robin fashion
    campaign_costs[campaign_id] += cost_per_communication[medium][i]

campaign_costs = {campaign_id: round(total_cost, 2) for campaign_id, total_cost in campaign_costs.items()} #round to 2 dp

#generate personalization cost with variability
#base costs for each personalization level
base_personalization_costs = {
    'low': 10.00,
    'medium': 25.00,
    'high': 50.00
}

#variability (20% of the base cost)
personalization_cost_noise = {
    'low': 2.00,
    'medium': 5.00,
    'high': 10.00
}

personalization_costs = np.zeros(num_customers)  #initialization
for i in range(num_customers):
    level = personalization_levels[i]
    base_cost = base_personalization_costs[level]
    noise = np.random.normal(0, personalization_cost_noise[level])  #random noise
    personalization_costs[i] = base_cost + noise
personalization_costs = np.round(personalization_costs, 2)  #round to 2dp

#no negative personalization costs (set a minimum cost threshold)
personalization_costs = np.clip(personalization_costs, 0, None)

#add personalization cost to campaign cost
for i in range(num_customers):
    campaign_id = campaign_ids[i % num_campaigns]  #assign customers to campaigns in a round-robin fashion
    campaign_costs[campaign_id] += personalization_costs[i]

#count the number of customers in each campaign
customers_per_campaign = {campaign_id: 0 for campaign_id in campaign_ids}
for i in range(num_customers):
    campaign_id = campaign_ids[i % num_campaigns]
    customers_per_campaign[campaign_id] += 1

#cost per customer for each campaign
cost_per_customer_by_campaign = {
    campaign_id: round(campaign_costs[campaign_id] / customers_per_campaign[campaign_id], 2)
    for campaign_id in campaign_ids
}

#generate open data (whether the customer opened the communication)
#higher personalization level, higher the probability of opening
open_rates = {
    'low': 0.2,
    'medium': 0.5,
    'high': 0.8
}

opens = np.zeros(num_customers, dtype=int)  #initialization
for i in range(num_customers):
    personalization_level = personalization_levels[i]
    opens[i] = np.random.choice([0, 1], p=[1 - open_rates[personalization_level], open_rates[personalization_level]])

#generate respond data (whether the customer took a specific action after opening)
#only applicable for customers who opened the communication (open = 1)
respond_rates = {
    'low': 0.2,
    'medium': 0.55,
    'high': 0.9
}

responds = np.zeros(num_customers, dtype=int)  # Initialization
for i in range(num_customers):
    if opens[i] == 1:  #only applicable for customers who opened the communication (open = 1)
        personalization_level = personalization_levels[i]
        responds[i] = np.random.choice([0, 1], p=[1 - respond_rates[personalization_level], respond_rates[personalization_level]])

#generate "deal" column, where only customers who responded (respond = 1) have a chance of securing a deal.
deal_probability = 0.7

#initialize "deal" column with 0 (no deal)
deals = np.zeros(num_customers, dtype=int)

#for customers who responded, randomly assign a deal with the given probability
for i in range(num_customers):
    if responds[i] == 1:  # Only consider customers who responded
        deals[i] = np.random.choice([0, 1], p=[1 - deal_probability, deal_probability])

#generate cost-related features
cost_per_customer = np.zeros(num_customers)

#assign campaign_cost and cost_per_customer based on campaign_id
campaign_id_list = np.random.choice(campaign_ids, size=num_customers)  #assign campaign_ids to customers
for i in range(num_customers):
    campaign_id = campaign_id_list[i]
    cost_per_customer[i] = cost_per_customer_by_campaign[campaign_id]  #cost per customer for the campaign

cost_per_customer = np.round(cost_per_customer, 2) #round cost_per_customer to 2 dp

#set the average revenue per dollar spent
average_revenue_per_dollar = 18.34

#variability
revenue_std = 15

#normal distribution with a high standard deviation for huge variance
revenue_variance_factor = np.random.normal(average_revenue_per_dollar, revenue_std, size=num_customers)

#initialize revenue column
revenue = np.zeros(num_customers)

#calculate revenue for customers who secured a deal (deal = 1)
for i in range(num_customers):
    if deals[i] == 1:  #for customers who secured a deal (deal = 1)
        revenue[i] = round(cost_per_customer[i] * revenue_variance_factor[i], 2)  #marketing spend * variability factor

roi = np.zeros(num_customers)  #initialize ROI column with 0s

#iterate over each customer and calculate ROI for those with a deal (deal = 1)
for i in range(num_customers):
    if deals[i] == 1:  #for customers who secured a deal (deal = 1)
        inter = revenue[i]
        cost = cost_per_customer[i]
        roi[i] = round((inter - cost) / cost, 2)  #ROI formula

#create the dataset
data = {
    'customer_id': customer_ids,
    'age': ages,
    'gender': genders,
    'marital_status': marital_statuses,
    'education' : education,
    'monthly_income': monthly_incomes,
    'bank_balance': bank_balances,
    'avg_transaction_value': avg_transaction_values,
    'transaction_frequency': transaction_frequencies,
    'campaign_id': campaign_id_list,
    'medium': mediums,
    'campaign_cost': [round(campaign_costs[cid], 2) for cid in campaign_id_list],
    'personalization_level': personalization_levels,
    'personalization_cost': personalization_costs,
    'open': opens,
    'respond': responds,
    'deal' : deals,
    'cost_per_customer': cost_per_customer,
    'revenue' : revenue,
    'roi' : roi
}

#convert to DataFrame
df = pd.DataFrame(data)

print(df.head())

#show total campaign costs
print("\nTotal campaign costs (including personalization):")
for campaign_id, total_cost in campaign_costs.items():
    print(f"Campaign {campaign_id}: ${total_cost:.2f}")

#show cost per customer for each campaign
print("\nCost per customer for each campaign:")
for campaign_id, cost in cost_per_customer_by_campaign.items():
    print(f"Campaign {campaign_id}: ${cost:.2f}")

#save the dataset to a CSV file
#df.to_csv('3101_data.csv', index=False)

                            customer_id  age  gender marital_status  \
0  109efb40-8ceb-4c57-97e3-23cf6d651847   56  female       divorced   
1  50d3802d-b876-4455-88b3-152d845a91b8   69    male        married   
2  c75ff368-5f7d-4bb1-880a-fe9f84a6d693   46    male        married   
3  9a7fde23-af59-4a66-a75b-21134f62cc9d   32  female         single   
4  7414e3e1-a387-4e92-9362-0119efa9a0a3   60    male        married   

     education  monthly_income  bank_balance  avg_transaction_value  \
0  high school         5961.23     100294.74                  78.25   
1     bachelor         6977.16     114052.66                 114.35   
2     bachelor         5431.66      91446.87                 125.26   
3     bachelor         3025.27      57748.44                 104.10   
4       master         6491.55      99132.77                 143.71   

   transaction_frequency  campaign_id       medium  campaign_cost  \
0                     55            2          SMS       56793.07   
1       

personalization_level
high      2664
low        672
medium    1709
Name: open, dtype: int64
Occurrences of each value in the 'open' column:
open
1    5045
0    4955
Name: count, dtype: int64

Occurrences of each value in the 'respond' column:
respond
0    6574
1    3426
Name: count, dtype: int64
Number of occurrences of each campaign_id:
campaign_id
5    2072
4    1999
1    1985
2    1981
3    1963
Name: count, dtype: int64
Average ROI for customers who secured a deal: 17.66
